# Modele MultiNomiale(nayev_bayeseinne)

## Importation des Donnes


In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [7]:
data=pd.read_csv('train_set.csv')
data=data.drop(['moyeneGeneral','Note1', 'Note2', 'Note3','Note4', 'Note5', 'Note6', 'Note7', 'Note8','Nom','ID'],axis=1)
data.head()

,Unnamed: 0,"Serie,x",GroupeAnonymat,LieuNaissance,Centre,Etablissement,Decision,Willaya,moughataa,DateNaissance
0,27634,SN,SN09,Bathet meit,Lycée Jedida,Lycée Jedida,Ajourné,Nouakchott 2 (Ouest),KSAR,12 ديسمبر 1998
1,36311,LO,LO1,Beir Taouress,Lycée Arafat 2 (A),Lycée Arafat 2,Ajourné,Nouakchott 3 (Sud),ARAFAT,26 ديسمبر 1991
2,21059,LM,LM3,Akjoujt,Lycée Akjoujt,Lycée Akjoujt,Ajourné,Inchiri,AKJOUJT,8 مايو 1995
3,24532,SN,SN06,Chelkhet Tiyab,Lycée Toujounine 1,Lycée Toujounine 1,Ajourné,Nouakchott 1 (Nord),TOUJOUNINE,24 avr. 1999
4,17861,SN,SN03,Aere M'Bar,Lycée Nouadhibou 2,Lycée Wava Nouadhibou,Ajourné,Dakhlet NDB,NOUADHIBOU,10 mai 2001


## Preprocessing

In [135]:
# Sélectionner les variables catégorielles
cat_vars = [ 'Serie,x', 'GroupeAnonymat', 'LieuNaissance', 'Centre', 'Etablissement', 'Willaya', 'moughataa','age']
cat_data = data[cat_vars].values
cat_data.shape

(23894, 8)

In [150]:
from sklearn.preprocessing import OneHotEncoder

# Instancier un objet OneHotEncoder
encoder = OneHotEncoder(sparse=False,handle_unknown='ignore')
# Encoder les variables catégorielles
X = encoder.fit_transform(cat_data)

In [151]:
X.shape

(23894, 1137)

In [10]:
### creer la variable age
data['age'] = data['DateNaissance'].apply(  
    lambda x: int(x.split("-")[2] if '-' in x else int(x.split(" ")[2]) )
    ).apply(lambda x : 2021 - x if x > 1900  else ( 121 - x if x > 21 else 21 - x))

In [ ]:
# Transformer les observations pour qu'on avoir deux modalites 'admis' et 'ajournee'
data['Decision'] = data['Decision'].replace(['Ajourné', 'Sessionnaire', 'Abscent', 'Examen annulé à cause du Téléphone', 'Examen annulé à cause du comportement'], 'Ajourné')

In [33]:
### encoder la variable target 
data['Decision']=pd.get_dummies(data['Decision'],drop_first=True)
data.head()

,Unnamed: 0,"Serie,x",GroupeAnonymat,LieuNaissance,Centre,Etablissement,Decision,Willaya,moughataa,DateNaissance,age
0,27634,SN,SN09,Bathet meit,Lycée Jedida,Lycée Jedida,1,Nouakchott 2 (Ouest),KSAR,12 ديسمبر 1998,23
1,36311,LO,LO1,Beir Taouress,Lycée Arafat 2 (A),Lycée Arafat 2,1,Nouakchott 3 (Sud),ARAFAT,26 ديسمبر 1991,30
2,21059,LM,LM3,Akjoujt,Lycée Akjoujt,Lycée Akjoujt,1,Inchiri,AKJOUJT,8 مايو 1995,26
3,24532,SN,SN06,Chelkhet Tiyab,Lycée Toujounine 1,Lycée Toujounine 1,1,Nouakchott 1 (Nord),TOUJOUNINE,24 avr. 1999,22
4,17861,SN,SN03,Aere M'Bar,Lycée Nouadhibou 2,Lycée Wava Nouadhibou,1,Dakhlet NDB,NOUADHIBOU,10 mai 2001,20


In [35]:
y = data['Decision']

In [152]:
# Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

## Entrainement du modele

In [153]:
# Entraînement du modèle de classification multinomiale Naive Bayes
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = clf.predict(X_test)

## Evaluation du modele

In [154]:
# Calcul des scores de précision, rappel et F1
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score:', accuracy_score(y_test, y_pred))
print('Precision score:', precision_score(y_test, y_pred))
print('Recall score:', recall_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred))

Accuracy score: 0.8462021343377275
Precision score: 0.9532756489493202
Recall score: 0.8759654702407996
F1 score: 0.9129868592399669


In [155]:
clf.score(X_test,y_test)

0.8462021343377275

## tester le modele sur les donnes de submission_template

### Creation du dataset new_data


In [48]:
test=pd.read_csv('test_set.csv',sep=';',index_col='ID')
test=test.drop(["Unnamed: 0",'prediction'],axis=1)

In [51]:
test.head()

,Nom,"Serie,x",GroupeAnonymat,LieuNaissance,Centre,Etablissement,Willaya,moughataa,DateNaissance,age
ID,,,,,,,,,,
ID_6,Mohamed Hama Ahmed Lely,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,15 يوليو 2001,20
ID_15,Fatimatou Mohamed Sid Abbe,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,12 août 1999,22
ID_18,El Alye Cheikh Ahmed Maihmatt,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,18 sept. 1999,22
ID_19,Haine Mohamed Lemin Telba,SN,SN09,Vassala,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,24 août 1998,23
ID_20,Aichata Sid'Ahmed Ahmed Ely,SN,SN09,El Megve,Lycée Vassala,Lycée Vassala,Hod Charghy,BASSEKNOU,31-mai-95,26


In [50]:
test['age'] = test['DateNaissance'].apply(  
    lambda x: int(x.split("-")[2] if '-' in x else int(x.split(" ")[2]) )
    ).apply(lambda x : 2021 - x if x > 1900  else ( 121 - x if x > 21 else 21 - x))

In [52]:
test=test.drop("DateNaissance",axis=1)

In [46]:
data1=pd.read_csv('train_set.csv',index_col='ID')
data1=data1.drop(["Unnamed: 0",'moyeneGeneral','Note1', 'Note2', 'Note3','Note4', 'Note5', 'Note6', 'Note7', 'Note8'],axis=1)
data1.head()

,Nom,"Serie,x",GroupeAnonymat,LieuNaissance,Centre,Etablissement,Decision,Willaya,moughataa,DateNaissance
ID,,,,,,,,,,
ID_27634,El Ide Brahim M'Hadi,SN,SN09,Bathet meit,Lycée Jedida,Lycée Jedida,Ajourné,Nouakchott 2 (Ouest),KSAR,12 ديسمبر 1998
ID_36311,Mariem Yedaly Afane,LO,LO1,Beir Taouress,Lycée Arafat 2 (A),Lycée Arafat 2,Ajourné,Nouakchott 3 (Sud),ARAFAT,26 ديسمبر 1991
ID_21059,Aiche Jiyid Jiyid,LM,LM3,Akjoujt,Lycée Akjoujt,Lycée Akjoujt,Ajourné,Inchiri,AKJOUJT,8 مايو 1995
ID_24532,Ahmed Abd El Ghader Meissa,SN,SN06,Chelkhet Tiyab,Lycée Toujounine 1,Lycée Toujounine 1,Ajourné,Nouakchott 1 (Nord),TOUJOUNINE,24 avr. 1999
ID_17861,Mariam Abdoulaye Ba,SN,SN03,Aere M'Bar,Lycée Nouadhibou 2,Lycée Wava Nouadhibou,Ajourné,Dakhlet NDB,NOUADHIBOU,10 mai 2001


In [53]:
data1['age'] = data1['DateNaissance'].apply(  
    lambda x: int(x.split("-")[2] if '-' in x else int(x.split(" ")[2]) )
    ).apply(lambda x : 2021 - x if x > 1900  else ( 121 - x if x > 21 else 21 - x))

In [54]:
data1=data1.drop("DateNaissance",axis=1)

In [57]:
data1=data1.drop("Decision",axis=1)

In [77]:
new=pd.concat([test,data1],axis=0)

In [80]:
new=new.reset_index()

In [109]:
new.shape

(15930, 10)

In [202]:
sub=pd.read_csv('submission_template.csv')

In [82]:
new = new[new.ID.isin(sub.Id)]

In [201]:
new.to_csv('new_data.csv')

In [145]:
cat_vars2 = [ 'Serie,x', 'GroupeAnonymat', 'LieuNaissance', 'Centre', 'Etablissement', 'Willaya', 'moughataa','age']
cat_data2 = new[cat_vars2].values

In [146]:
cat_data2.shape

(15930, 8)

In [156]:
# encoder les variables categorielles 
X2 = encoder.transform(cat_data2)

In [148]:
X2.shape


(15930, 1042)

In [157]:
y_pred2 = clf.predict(X2)


In [158]:
y_pred2
g=pd.DataFrame(y_pred2)
g.value_counts()

1    13431
0     2499
dtype: int64

In [168]:
y_pred2

array([1, 1, 1, ..., 1, 1, 1], dtype=uint8)

In [159]:
#trouver la probabilite d'admissiom
proba=clf.predict_proba(X2)

In [160]:
proba

array([[2.41599113e-03, 9.97584009e-01],
       [1.23271857e-02, 9.87672814e-01],
       [1.23271857e-02, 9.87672814e-01],
       ...,
       [2.10054629e-03, 9.97899454e-01],
       [1.38103468e-04, 9.99861897e-01],
       [5.03757574e-04, 9.99496242e-01]])

In [164]:
pre=proba[:, 0]
len(pre)

15930

In [165]:
new['prediction']=pre

C:\Users\TOSHIBA\AppData\Local\Temp\ipykernel_6372\1801944449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['prediction']=pre


In [187]:
submission=new[['ID','prediction']]

In [170]:
submission.to_csv('sub_nayev_bayesien.csv')

In [171]:
submission.shape

(15930, 2)

In [194]:
r=pd.read_csv('sub_nayev_bayesien.csv',sep=';')

In [198]:
r.to_csv('sub_nayev_bayesiennn.csv',index=False)

In [199]:
r

,Id,Predicted
0,ID_15,0.002416
1,ID_22,0.012327
2,ID_23,0.012327
3,ID_29,0.003012
4,ID_34,0.003012
...,...,...
15925,ID_13569,0.000026
15926,ID_31670,0.000108
15927,ID_31759,0.002101
15928,ID_5855,0.000138
